In [1]:
import numpy as np
import pandas as pd

In [2]:
ratings_df = pd.read_csv('ratings.csv')

In [3]:
ratings_df.head()

,userId,outfitID,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [4]:
userInput = [
            {'outfitID':1, 'rating':5},
            {'outfitID':2, 'rating':3.5},
            {'outfitID':296, 'rating':2},
            {'outfitID':1274, 'rating':5},
            {'outfitID':1968, 'rating':4.5}
         ] 
inputOutfit = pd.DataFrame(userInput)
inputOutfit

,outfitID,rating
0,1,5.0
1,2,3.5
2,296,2.0
3,1274,5.0
4,1968,4.5


In [5]:
inputOutfit.dtypes

outfitID      int64
rating      float64
dtype: object

In [6]:
userSubset = ratings_df[ratings_df['outfitID'].isin(inputOutfit['outfitID'].tolist())]
userSubset.head()

,userId,outfitID,rating
0,1,1,4.0
16,1,296,3.0
320,4,296,1.0
422,4,1968,4.0
516,5,1,4.0


In [7]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

In [8]:
userSubsetGroup.get_group(4)

,userId,outfitID,rating
320,4,296,1.0
422,4,1968,4.0


In [9]:
len(userSubsetGroup.get_group(4))

2